In [1]:
from bs4 import BeautifulSoup

In [2]:
import pandas as pd

In [3]:
import re

In [7]:
import requests

In [8]:
url = "https://www.bfro.net/gdb/"
raw_html = requests.get(url).content

In [9]:
page = BeautifulSoup(raw_html, 'html.parser')

In [10]:
us_table = page.find_all('table')

In [11]:
table_content = us_table[0].find_all('table')[2:]

In [12]:
table_content[3].text

'Province# ofListingsMostRecentReportLastPostedAlberta433-20207-2018British Columbia13512-202012-2020Manitoba318-20121-2011New Brunswick812-20218-2021Province# ofListingsMostRecentReportLastPostedNova Scotia311-201610-2015Ontario688-20207-2020Quebec44-20081-2007Saskatchewan710-20084-2006Yukon55-20093-2009'

In [13]:
data = []
base = 'https://www.bfro.net'
for tables in table_content[1:]:
    rows = tables.find_all('tr')[1:]
    for row in rows:
        cols = row.find_all('td')
        state = cols[0].text.strip()
        a_tag = cols[0].find('a')
        link = base + a_tag['href'] if a_tag else None
        num_listings = cols[1].text.strip()
        most_recent_report = cols[2].text.strip()
        last_posted = cols[3].text.strip()
        data.append([state, num_listings, most_recent_report, last_posted,link])

In [14]:
df = pd.DataFrame(data, columns=["State/Province", "# of Listings", "Most Recent Report", "Last Posted","Link"])

# GoogleGeoAPI to find the coordinate

In [18]:
import time

In [238]:
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
import os
api_key = os.environ.get('GMAP_API')

In [14]:
placeNames = df['State/Province']

In [15]:
url = 'https://maps.googleapis.com/maps/api/geocode/json'
keys = {'address': 'Alaska', 'key': api_key}
r = requests.get(url,params=keys)
result_dic = r.json()

In [16]:
result_dic

{'results': [{'address_components': [{'long_name': 'Alaska',
     'short_name': 'AK',
     'types': ['administrative_area_level_1', 'political']},
    {'long_name': 'United States',
     'short_name': 'US',
     'types': ['country', 'political']}],
   'formatted_address': 'Alaska, USA',
   'geometry': {'bounds': {'northeast': {'lat': 71.439786,
      'lng': -129.9741939},
     'southwest': {'lat': 51.175092, 'lng': 172.347846}},
    'location': {'lat': 63.588753, 'lng': -154.4930619},
    'location_type': 'APPROXIMATE',
    'viewport': {'northeast': {'lat': 71.439786, 'lng': -129.9741939},
     'southwest': {'lat': 51.175092, 'lng': 172.347846}}},
   'place_id': 'ChIJG8CuwJzfAFQRNduKqSde27w',
   'types': ['administrative_area_level_1', 'political']}],
 'status': 'OK'}

In [28]:
geometry_df = []
#looping through the placeNames list
for place in placeNames:
    url = 'https://maps.googleapis.com/maps/api/geocode/json'
    keys = {'address': place, 'key': api_key}
    r = requests.get(url,params=keys)
    result_dic = r.json()
    if 'results' in result_dic and len(result_dic['results']) > 0:
        lat = result_dic['results'][0]['geometry']['location']['lat']
        long = result_dic['results'][0]['geometry']['location']['lng']
        coords = []
        coords.append(long)
        coords.append(lat)
        mygeometry = {'place': place, 'geometry.type': 'Point','geometry.coordinates':coords,'lat':lat,'long':long}
        geometry_df.append(mygeometry)
    else:
        print("No results found. Check your query or API key.")
    print(mygeometry)
    time.sleep(0.5)

{'place': 'Alaska', 'geometry.type': 'Point', 'geometry.coordinates': [-154.4930619, 63.588753]}
{'place': 'Alabama', 'geometry.type': 'Point', 'geometry.coordinates': [-86.902298, 32.3182314]}
{'place': 'Arkansas', 'geometry.type': 'Point', 'geometry.coordinates': [-91.8318334, 35.20105]}
{'place': 'Arizona', 'geometry.type': 'Point', 'geometry.coordinates': [-111.0937311, 34.0489281]}
{'place': 'California', 'geometry.type': 'Point', 'geometry.coordinates': [-119.4179324, 36.778261]}
{'place': 'Colorado', 'geometry.type': 'Point', 'geometry.coordinates': [-105.7820674, 39.5500507]}
{'place': 'Connecticut', 'geometry.type': 'Point', 'geometry.coordinates': [-73.087749, 41.6032207]}
{'place': 'Delaware', 'geometry.type': 'Point', 'geometry.coordinates': [-75.52766989999999, 38.9108325]}
{'place': 'Florida', 'geometry.type': 'Point', 'geometry.coordinates': [-81.5157535, 27.6648274]}
{'place': 'Georgia', 'geometry.type': 'Point', 'geometry.coordinates': [43.35689199999999, 42.315407]}
{

In [31]:
geometry = pd.DataFrame(geometry_df)

In [35]:
geometry.head()

,place,geometry.type,geometry.coordinates
0,Alaska,Point,"[-154.4930619, 63.588753]"
1,Alabama,Point,"[-86.902298, 32.3182314]"
2,Arkansas,Point,"[-91.8318334, 35.20105]"
3,Arizona,Point,"[-111.0937311, 34.0489281]"
4,California,Point,"[-119.4179324, 36.778261]"


In [33]:
geometry.to_csv('Bigfoot_geo.csv',index=False)

In [17]:
merged = df.merge(geometry, left_on='State/Province', right_on='place')

In [18]:
merged.head()

,State/Province,# of Listings,Most Recent Report,Last Posted,Link,place,geometry.type,geometry.coordinates
0,Alaska,22,12-2022,5-2022,https://www.bfro.net/GDB/state_listing.asp?sta...,Alaska,Point,"[-154.4930619, 63.588753]"
1,Alabama,102,2-2023,2-2023,https://www.bfro.net/GDB/state_listing.asp?sta...,Alabama,Point,"[-86.902298, 32.3182314]"
2,Arkansas,111,3-2023,9-2022,https://www.bfro.net/GDB/state_listing.asp?sta...,Arkansas,Point,"[-91.8318334, 35.20105]"
3,Arizona,85,6-2021,6-2021,https://www.bfro.net/GDB/state_listing.asp?sta...,Arizona,Point,"[-111.0937311, 34.0489281]"
4,California,463,10-2024,10-2024,https://www.bfro.net/GDB/state_listing.asp?sta...,California,Point,"[-119.4179324, 36.778261]"


In [21]:
merged.to_csv('geo_coord.csv')

# Scrape County level data for US

In [92]:
us_states = merged[:49]

In [106]:
data = []
base = 'https://www.bfro.net/GDB/'
for url in us_states['Link']:
    raw_html = requests.get(url).content
    html = BeautifulSoup(raw_html,'html.parser')
    counties = html.find_all('table')[3].find_all('table')[1].find_all('tr')[0]
    for c in counties:
        rows = counties.find_all('tr')[1:]
        for row in rows:
            cols = row.find_all('td')
            if len(cols) <4:
                continue
            county_tag = cols[0].find('a')
            if county_tag:
                county = county_tag.text
                link = base+county_tag.get('href')
            else:
                county = cols[0].text.strip()
                link = None
    
            num_listings = cols[1].text.strip()
            most_recent_report = cols[2].text.strip()
            last_posted = cols[3].text.strip()
            data.append([county, num_listings, most_recent_report, last_posted,link])

In [107]:
us_counties = pd.DataFrame(data,columns=["Counties", "# of Listings", "Most Recent Report", "Last Posted","Link"])
us_counties.head()

,Counties,# of Listings,Most Recent Report,Last Posted,Link
0,Aleutian Islands East,,,,None
1,Aleutian Islands West,,,,None
2,Anchorage,2,2-2004,11-2005,https://www.bfro.net/GDB/show_county_reports.a...
3,Angoon,,,,None
4,Barrow,,,,None


In [137]:
us_counties.to_csv('us_counties.csv',index= False)

# Scrape Class A/B 

In [140]:
url = 'https://www.bfro.net/GDB/show_county_reports.asp?state=ak&county=Prince+of+Wales'
raw_html = requests.get(url).content
html = BeautifulSoup(raw_html,'html.parser')

In [141]:
county = html.find('h2').text

In [143]:
print(county)

Prince of Wales Borough, Alaska


In [112]:
incidents = html.find('ul').find_all('li')
incidents

[<li class="spaced"><span class="reportcaption"><b><a href="show_report.asp?id=7963">February 2004</a></b></span> <span class="highlighted">(Class A)</span><span class="reportcaption"> - Sasquatch walks past window of house at night</span>
 <li class="spaced"><span class="reportcaption"><b><a href="show_report.asp?id=8797">May 2001</a></b></span> <span class="highlighted">(Class B)</span><span class="reportcaption"> - Two boys find footprints on Harris River bank</span>
 <li class="spaced"><span class="reportcaption"><b><a href="show_report.asp?id=2917">Spring 1995</a></b></span> <span class="highlighted">(Class A)</span><span class="reportcaption"> - Family observes large biped from car</span>
 </li></li></li>,
 <li class="spaced"><span class="reportcaption"><b><a href="show_report.asp?id=8797">May 2001</a></b></span> <span class="highlighted">(Class B)</span><span class="reportcaption"> - Two boys find footprints on Harris River bank</span>
 <li class="spaced"><span class="reportcapt

In [122]:
counties = us_counties.dropna()['Link']

In [144]:
d = []
for l in counties:
    try:
        raw_html = requests.get(l).content
        html = BeautifulSoup(raw_html,'html.parser')
        county = html.find('h2').text
        incidents = html.find('ul').find_all('li')
        for i in incidents:
            try:
                cols = i.find_all('span')
                date = cols[0].text
                level = cols[1].text
                description = cols[2].text
                d.append([county, date, level, description])
            except:
                print(f"{i} not found.")
    except:
            print(f"{l} not found.")

<li class="spaced"><span class="reportcaption"><b><a href="show_article.asp?id=576" ttarget="_blank">Hairy Wild Man Sought In Swamp</a></b></span><span class="reportcaption"> (15-Apr-1938; The Oshkosh Northwestern)</span></li> not found.
<li class="spaced"><span class="reportcaption"><b><a href="show_article.asp?id=576" ttarget="_blank">Hairy Wild Man Sought In Swamp</a></b></span><span class="reportcaption"> (15-Apr-1938; The Oshkosh Northwestern)</span></li> not found.
<li class="spaced"><span class="reportcaption"><b><a href="show_article.asp?id=709" ttarget="_blank">The Legend of the Skunk Ape</a></b></span><span class="reportcaption"> (24-May-2014; WCJB - ABC News)</span></li> not found.
<li class="spaced"><span class="reportcaption"><b><a href="show_article.asp?id=709" ttarget="_blank">The Legend of the Skunk Ape</a></b></span><span class="reportcaption"> (24-May-2014; WCJB - ABC News)</span></li> not found.
<li class="spaced"><span class="reportcaption"><b><a href="show_article.

In [145]:
US_classes = pd.DataFrame(d,columns= ['Place','Date','Class','Description'])

In [146]:
US_classes.to_csv('US_classes.csv',index=False)

# Scrape the rest

In [151]:
rest = merged[49:]

In [157]:
Links = rest.drop_duplicates()['Link']

In [158]:
d = []
for l in Links:
    try:
        raw_html = requests.get(l).content
        html = BeautifulSoup(raw_html,'html.parser')
        county = html.find('h2').text
        incidents = html.find('ul').find_all('li')
        for i in incidents:
            try:
                cols = i.find_all('span')
                date = cols[0].text
                level = cols[1].text
                description = cols[2].text
                d.append([county, date, level, description])
            except:
                print(f"{i} not found.")
    except:
            print(f"{l} not found.")

<li class="spaced"><span class="reportcaption"><b><a href="show_article.asp?id=407" ttarget="_blank">Yowie</a></b></span><span class="reportcaption"> (13-Jun-2002; ABC News)</span></li> not found.
<li class="spaced"><span class="reportcaption"><b><a href="show_article.asp?id=221" ttarget="_blank">9-foot, Redheaded Vegetarian Stalks China</a></b></span><span class="reportcaption"> (26-Mar-1981; Detroit Free press)</span></li> not found.
<li class="spaced"><span class="reportcaption"><b><a href="show_article.asp?id=254" ttarget="_blank">Team 'find traces of Sumatran Yeti'</a></b></span><span class="reportcaption"> (27-Oct-2001; The Times (London, UK))</span></li> not found.
<li class="spaced"><span class="reportcaption"><b><a href="show_article.asp?id=212" ttarget="_blank">Malaysian trekkers report fresh footprints of 'Bigfoot'</a></b></span><span class="reportcaption"> (13-Jan-1995; Deseret News)</span></li> not found.
<li class="spaced"><span class="reportcaption"><b><a href="show_arti

In [160]:
rest_counties = pd.DataFrame(d,columns=['Place','Date','Class','Description'])

In [161]:
rest_counties.head()

,Place,Date,Class,Description
0,Alberta (Canada),July 2018,(Class B),- Hiker finds possible tracks in Indian Ridge...
1,Alberta (Canada),June 2018,(Class B),- Retired mail carrier reports sundown knocks...
2,Alberta (Canada),September 2016,(Class B),- Two hunters experience prolonged tree shaki...
3,Alberta (Canada),June 2011,(Class A),- Early morning sighting through binoculars w...
4,Alberta (Canada),May 2011,(Class B),- Possible dusk sighting at Kananaskis campgr...


In [162]:
all_classes = pd.concat([US_classes, rest_counties], ignore_index=True)

In [164]:
all_classes.describe()

,Place,Date,Class,Description
count,10687,10687,10687,10687
unique,1574,1421,82,5504
top,"Pierce County, Washington",July 2006,(Class B),- Loud vocalizations sounds heard
freq,158,68,5323,4


In [165]:
all_classes.to_csv('all_classes.csv',index=False)

In [167]:
all_classes.head(-20)

,Place,Date,Class,Description
0,"Anchorage Borough, Alaska",February 2004,(Class A),- Snowmobiler has encounter in deep snow near...
1,"Anchorage Borough, Alaska",December 2003,(Class B),- Four nocturnal hikers get pelted with snow ...
2,"Bethel Borough, Alaska",September 1998,(Class B),- Creature observed walking back and forth by...
3,"Bristol Bay Borough, Alaska",July 2004,(Class B),- Fishermen find footprints east of Egegik.
4,"Cordova-McCarthy Borough, Alaska",June 2000,(Class A),- Campers' encounter just after dark in the W...
...,...,...,...,...
10662,Malaysia (International),[Investigators in Kota Tinggi],(22-Feb-2006; New Straits Times),[Sightings in Endau-Rompin National Park]
10663,Malaysia (International),[Sightings in Endau-Rompin National Park],(20-Feb-2006; Bernama (Malaysia National News...,[Malaysia effort will likely uncover other new...
10664,Malaysia (International),[Malaysia effort will likely uncover other new...,(13-Feb-2006; Bernama (Malaysia National News...,[Malaysian] Panel to co-ordinate Bigfoot exp...
10665,Malaysia (International),[Malaysian] Panel to co-ordinate Bigfoot exp...,(11-Feb-2006; New Straits Times - Malaysia),[Malaysia forms official scientific team]


# clean data

In [180]:
dt = all_classes[all_classes['Class'].isin(['(Class A)','(Class B)'])]

In [182]:
dt.describe()

,Place,Date,Class,Description
count,10519,10519,10519,10519
unique,1554,1332,2,5387
top,"Pierce County, Washington",July 2006,(Class B),- Loud vocalizations sounds heard
freq,158,68,5323,4


In [183]:
dt.head()

,Place,Date,Class,Description
0,"Anchorage Borough, Alaska",February 2004,(Class A),- Snowmobiler has encounter in deep snow near...
1,"Anchorage Borough, Alaska",December 2003,(Class B),- Four nocturnal hikers get pelted with snow ...
2,"Bethel Borough, Alaska",September 1998,(Class B),- Creature observed walking back and forth by...
3,"Bristol Bay Borough, Alaska",July 2004,(Class B),- Fishermen find footprints east of Egegik.
4,"Cordova-McCarthy Borough, Alaska",June 2000,(Class A),- Campers' encounter just after dark in the W...


In [188]:
import re
y = r"(\b\d{4})"

In [189]:
years = dt['Date'].str.extract(y)
dt['Year'] = years

In [194]:
c = r"[(](Class \w)[)]"
classes = dt['Class'].str.extract(c)
dt['Class'] = classes

In [202]:
Place = dt['Place'].str.replace(r"[()]", "", regex=True)
dt['Place'] = Place

In [205]:
dt.head()

,Place,Date,Class,Description,Year
0,"Anchorage Borough, Alaska",February 2004,Class A,- Snowmobiler has encounter in deep snow near...,2004
1,"Anchorage Borough, Alaska",December 2003,Class B,- Four nocturnal hikers get pelted with snow ...,2003
2,"Bethel Borough, Alaska",September 1998,Class B,- Creature observed walking back and forth by...,1998
3,"Bristol Bay Borough, Alaska",July 2004,Class B,- Fishermen find footprints east of Egegik.,2004
4,"Cordova-McCarthy Borough, Alaska",June 2000,Class A,- Campers' encounter just after dark in the W...,2000


In [216]:
d = dt[['Class','Year']]
d

,Class,Year
0,Class A,2004
1,Class B,2003
2,Class B,1998
3,Class B,2004
4,Class A,2000
...,...,...
10622,Class B,2001
10623,Class B,1995
10624,Class A,1992
10625,Class B,1995


In [224]:
d.groupby('Year')['Class'].value_counts()

In [226]:
d.groupby('Year')['Class'].value_counts().unstack().to_clipboard()

# Find coord for all_classes

In [347]:
import time

In [316]:
points = dt['Place']

In [340]:
load_dotenv()
api_key = os.environ.get('GMAP_API')

In [349]:
geometry_dt = []
#looping through the placeNames list
for place in points:
    url = 'https://maps.googleapis.com/maps/api/geocode/json'
    keys = {'address': place, 'key': api_key}
    r = requests.get(url,params=keys)
    result_dic = r.json()
    if 'results' in result_dic and len(result_dic['results']) > 0:
        lat = result_dic['results'][0]['geometry']['location']['lat']
        long = result_dic['results'][0]['geometry']['location']['lng']
        coords = str(lat) + ', ' + str(long)
        mygeometry = {'place': place, 'geometry.type': 'Point','geometry.coordinates':coords,'lat':lat,'long':long}
        geometry_dt.append(mygeometry)
    else:
        print("No results found. Check your query or API key.")
    continue
    time.sleep(0.5)

In [352]:
county_geo = pd.DataFrame(geometry_dt)
county_geo.head()

,place,geometry.type,geometry.coordinates,lat,long
0,"Anchorage Borough, Alaska",Point,"61.2175758, -149.8996785",61.217576,-149.899678
1,"Anchorage Borough, Alaska",Point,"61.2175758, -149.8996785",61.217576,-149.899678
2,"Bethel Borough, Alaska",Point,"60.79425899999999, -161.7694402",60.794259,-161.769440
3,"Bristol Bay Borough, Alaska",Point,"58.7578504, -156.7792687",58.757850,-156.779269
4,"Cordova-McCarthy Borough, Alaska",Point,"60.5424087, -145.7525414",60.542409,-145.752541


## Find Elevation

In [402]:
pip install python-dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Users/yirenlu/.pyenv/versions/3.12.7/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [407]:
coords = county_geo['geometry.coordinates']

In [408]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [413]:
key = os.getenv('ElEVATION_API')

In [416]:
elevation = []
#looping through the placeNames list
for place in coords:
    url = 'https://maps.googleapis.com/maps/api/elevation/json'
    keys = {'locations': place, 'key': key}
    r = requests.get(url,params=keys)
    result_dic = r.json()
    if 'results' in result_dic and len(result_dic['results']) > 0:
        elev = result_dic['results'][0]['elevation']
        elevation.append(elev)
    else:
        print("No results found. Check your query or API key.")
    continue
    time.sleep(0.5)

In [418]:
county_geo['elevation'] = elevation

In [421]:
county_geo = county_geo.merge(dt,right_on = 'Place',left_on = 'place')

In [423]:
county_geo.head()

,place,geometry.type,geometry.coordinates,lat,long,elevation,Place,Date,Class,Description,Year
0,"Allen Parish, Louisiana",Point,"30.5887484, -92.8577105",30.588748,-92.857710,12.051091,"Allen Parish, Louisiana",October 1980,Class B,"- While hunting with his dog, a hunter observ...",1980
1,"Allen Parish, Louisiana",Point,"30.5887484, -92.8577105",30.588748,-92.857710,12.051091,"Allen Parish, Louisiana",October 1980,Class B,"- While hunting with his dog, a hunter observ...",1980
2,"Ascension Parish, Louisiana",Point,"30.2017295, -90.9438468",30.201729,-90.943847,2.743345,"Ascension Parish, Louisiana",October 2000,Class A,- Early evening sighting by motorist near Sor...,2000
3,"Ascension Parish, Louisiana",Point,"30.2017295, -90.9438468",30.201729,-90.943847,2.743345,"Ascension Parish, Louisiana",October 2000,Class A,- Early evening sighting by motorist near Sor...,2000
4,"Avoyelles Parish, Louisiana",Point,"30.9711824, -92.0665197",30.971182,-92.066520,15.280701,"Avoyelles Parish, Louisiana",Fall 1925,Class B,- Woman recounts a tale her Grandfather told ...,1925


In [425]:
class_plot = county_geo[['Place','Class','lat','long']]

In [427]:
class_plot.to_clipboard()